In [1]:
# importar bibliotecas

import PyPDF2
import re
import sys
sys.path.insert(0, 'C:\\Users\\MatheusPereira\\OneDrive - Pontte\\Área de Trabalho\\automacaoRegistroCCI\\Scripts')
from lerContrato import lerContrato
import lerContrato as lc

In [2]:
contratoPath = r'C:\Users\MatheusPereira\OneDrive - Pontte\Área de Trabalho\automacaoRegistroCCI\Contratos\HE_Contrato_Oliboni _Assinatura Digital_VFinal.pdf'

In [3]:
contrato = lerContrato(contratoPath)
quantidade = lc.dadosParticipantes(contratoPath, contrato)
contrato

{'valorTotal': '3201801.53',
 'tabela': 'SAC',
 'registro': '10554.15',
 'Taxa ao Mes': 0.1268,
 'valorLiquido': '2981000.00',
 'prazoMes': '228',
 'valorPrimeiraParcelaComEncargos': '48642.95',
 'valorImóvel': '6188000.00',
 'prazoContrato': '225',
 'ultimaParcela': '15/06/2041',
 'dataLiberação': '28/06/2022',
 'valorPrimeiraParcela': '47494.30',
 'primeiraParcela': '15/10/2022',
 'indice': 'IPCA',
 'Matrícula': '24515',
 'Cartório': '02 de Registro Geral do 9º Oficial de Registro de Imóveis da Comarca de Niterói/RJ',
 'CCI': '0001114017/OPS',
 'Titular': 'OLIBONI PARTICIPAÇÕES S/A ',
 'operação': 'PJ',
 'Participante1': 'ALENCAR LUIZ OLIBONI',
 'Participação1': '33%',
 'Participante2': 'LEONARDO COLODA OLIBONI',
 'Participação2': '33%',
 'Participante3': 'MARINA COLODA OLIBONI',
 'Participação3': '33%',
 'dataContrato': '24/06/2022'}

In [4]:
#Faz a leitura usando a biblioteca
pdf_file = open(contratoPath, 'rb')
read_pdf = PyPDF2.PdfFileReader(pdf_file)

# pega o numero de páginas
number_of_pages = read_pdf.getNumPages()

#Extriar Texto Página 1 a 5
text=''
for i in range(0,9):
    #Ler Página PDF
    pageObj = read_pdf.getPage(i)
    #Extrair Texto
    text=text+pageObj.extractText()

#Tratar Texto (Remover Quebra de Linhas e espaços duplos)
text = re.sub('\r', '', text) 
text = re.sub('\n', '', text)
text = re.sub(' {2,}', ' ', text).strip(' ')


In [5]:
if "HE_" in contratoPath:
    listaDePara = {'cpf':'CPF:','data de nascimento':'DATA DE NASCIMENTO:','endereço':'ENDEREÇO RESIDENCIAL E DOMICILIAR:','complemento':'COMPLEMENTO:','bairro':'BAIRRO:','cidade':'CIDADE:','uf':'UF:','cep':'CEP:',
                'telefone':'TELEFONE(S)','email':'EMAIL:'}

    #len(listaHE) # <--- Qtd de Itens na Lista
    listaKey = []
    listaValues = []

    # Remover cabeçalho da pg
    def removerText(valor):
        # Extraindo participantes da operação 
        começo = 'CONTRATO DE EMPRÉSTIMO'                        #inicio e fim da extração
        fim = 'Física Página'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = valor.find(começo, 0)
        finalTopico = valor.find(fim, 0)

        #Criar Paragráfo Auxiliar
        remover = valor[inicioTopico:finalTopico+len(fim)+8]
        remover = re.sub('\s+',' ', remover)
        return remover
        
    

    for qtdParticipantes in range(0, quantidade['Quantidade']):
        # Extraindo participantes da operação 
        começo = contrato[f'Participante{qtdParticipantes+1}']                        #inicio e fim da extração
        fim = 'CAMPO 3 -'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(começo, 0)
        finalTopico = text.find(fim, 0)

        #Criar Paragráfo Auxiliar
        paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
        paragrafoAux = re.sub('\s+',' ', paragrafoAux)

        for key, value in listaDePara.items():
            if key == 'endereço':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'complemento':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("BAIRRO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'bairro':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("CIDADE:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'cidade':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("UF:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            elif key == 'telefone':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("EMAIL:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            else:
                paragrafoAux = paragrafoAux + ' '
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find(" ", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            listaKey.append(f'{key}{qtdParticipantes+1}')
            listaValues.append(valorExtraido.strip())
elif "FI_" in contratoPath:
    listaDePara = {'cpf':'CPF:','data de nascimento':'DATA DE NASCIMENTO:','endereço':'ENDEREÇO RESIDENCIAL E DOMICILIAR:','complemento':'COMPLEMENTO:','bairro':'BAIRRO:','cidade':'CIDADE:','uf':'UF:','cep':'CEP:',
                'telefone':'TELEFONE(S)','email':'EMAIL:'}

    #len(listaHE) # <--- Qtd de Itens na Lista
    listaKey = []
    listaValues = []
    for qtdParticipantes in range(0, quantidade):
        # Extraindo participantes da operação 
        começo = contrato[f'Participante{qtdParticipantes+1}']                        #inicio e fim da extração
        fim = 'CARACTERÍSTICAS DO FINANCIAMENTO'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(começo, 0)
        finalTopico = text.find(fim, 0)

        #Criar Paragráfo Auxiliar
        paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
        paragrafoAux = re.sub('\s+',' ', paragrafoAux)
        paragrafoAux
        for key, value in listaDePara.items():
            if key == 'endereço':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'complemento':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("BAIRRO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'bairro':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("CIDADE:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'cidade':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("UF:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'telefone':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("EMAIL:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco].strip()
            else:
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find(" ", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]

            listaKey.append(f'{key}{qtdParticipantes+1}')
            listaValues.append(valorExtraido.strip())
#Criar Dicionario das duas Listas
dict_keyValue = dict(zip(listaKey,listaValues))
dict_keyValue


{'cpf1': '337.550.780-15',
 'data de nascimento1': '18/06/1961',
 'endereço1': 'AVENIDA RUI BARBOSA, 12',
 'complemento1': 'APTO 501',
 'bairro1': 'SÃO FRANCISCO',
 'cidade1': 'NITERÓI',
 'uf1': 'RJ',
 'cep1': '24360-440',
 'telefone1': '(21) 99983 2620',
 'email1': 'alencaroliboni@terra.com.br',
 'cpf2': '128.225.257-73',
 'data de nascimento2': '05/03/1990',
 'endereço2': 'RUA EURICO MANUEL DO CARMO, 02',
 'complemento2': 'BL. 2, AP 304',
 'bairro2': 'CHARITAS',
 'cidade2': 'NITERÓI',
 'uf2': 'RJ',
 'cep2': '24370090',
 'telefone2': '(21) 99983 2620',
 'email2': 'alencaroliboni@terra.com.br',
 'cpf3': '124.246.777-78',
 'data de nascimento3': '08/12/1988',
 'endereço3': 'RUA AMORA, 501',
 'complemento3': 'CASA G17',
 'bairro3': 'PIRATININGA',
 'cidade3': 'NITERÓI',
 'uf3': 'RJ',
 'cep3': '24350-590',
 'telefone3': '(21) 99983 2620',
 'email3': 'alencaroliboni@terra.com.br'}

In [7]:
# Estrair dados do Titular

if "HE_" in contratoPath:
    listaDePara = {'endereçoTitular':'ENDEREÇO COMERCIAL:','complementoTitular':'COMPLEMENTO:','bairroTitular':'BAIRRO:','cidadeTitular':'CIDADE:','ufTitular':'UF:','cepTitular':'CEP:','cnpjTitular':'CNPJ:'}
    # Extraindo participantes da operação 
    começo = contrato['Titular'].strip()                    #inicio e fim da extração
    fim = 'CARACTERÍSTICAS DO FINANCIAMENTO'
    #Pegar posição das variáveis auxiliares no texto
    inicioTopico = text.find(começo, 0)
    finalTopico = text.find(fim, 0)

    #Criar Paragráfo Auxiliar
    paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
    paragrafoAux = re.sub('\s+',' ', paragrafoAux)
    if contrato['operação'] == 'PJ':
        for key, value in listaDePara.items():
            if key == 'endereçoTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'complementoTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("BAIRRO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'bairroTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("CIDADE:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'cidadeTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("UF:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            else:
                paragrafoAux = paragrafoAux + ' '
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find(" ", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
        
            listaKey.append(key)
            listaValues.append(valorExtraido.strip())
#Criar Dicionario das duas Listas
dict_keyValue = dict(zip(listaKey,listaValues))
dict_keyValue


{'cpf1': '337.550.780-15',
 'data de nascimento1': '18/06/1961',
 'endereço1': 'AVENIDA RUI BARBOSA, 12',
 'complemento1': 'APTO 501',
 'bairro1': 'SÃO FRANCISCO',
 'cidade1': 'NITERÓI',
 'uf1': 'RJ',
 'cep1': '24360-440',
 'telefone1': '(21) 99983 2620',
 'email1': 'alencaroliboni@terra.com.br',
 'cpf2': '128.225.257-73',
 'data de nascimento2': '05/03/1990',
 'endereço2': 'RUA EURICO MANUEL DO CARMO, 02',
 'complemento2': 'BL. 2, AP 304',
 'bairro2': 'CHARITAS',
 'cidade2': 'NITERÓI',
 'uf2': 'RJ',
 'cep2': '24370090',
 'telefone2': '(21) 99983 2620',
 'email2': 'alencaroliboni@terra.com.br',
 'cpf3': '124.246.777-78',
 'data de nascimento3': '08/12/1988',
 'endereço3': 'RUA AMORA, 501',
 'complemento3': 'CASA G17',
 'bairro3': 'PIRATININGA',
 'cidade3': 'NITERÓI',
 'uf3': 'RJ',
 'cep3': '24350-590',
 'telefone3': '(21) 99983 2620',
 'email3': 'alencaroliboni@terra.com.br',
 'endereçoTitular3': 'Avenida Rui Barbosa, nº 158',
 'complementoTitular3': 'Loja Parte',
 'bairroTitular3': 'S

In [ ]:
for key, value in listaDePara.items():
    if key == 'endereçoTitular':
        inicioFrase = paragrafoAux.find(value,0)
        finalFrase = inicioFrase + len(value) + 1
        proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
        valorExtraido = paragrafoAux[finalFrase:proximoEspaco]